# Reinforcement Learning: Reward Shaping and Actor-Critic

Este notebook mostra dois exemplos didáticos:
1. **Reward Shaping** em Gridworld com implementação em NumPy.
2. **Actor-Critic (A2C)** usando Stable-Baselines3 em CartPole.

## Parte 1: Reward Shaping em Gridworld

In [1]:

import numpy as np

# Gridworld simples 4x4
n_states = 16
n_actions = 4  # up, right, down, left
gamma = 0.9

# Função para converter estado em (linha,coluna)
def to_pos(s):
    return divmod(s, 4)

# Movimentos
actions = {0: (-1,0), 1: (0,1), 2: (1,0), 3: (0,-1)}

# Estado terminal
terminal = [0, 15]

def step(s, a):
    if s in terminal:
        return s, 0
    r,c = to_pos(s)
    dr,dc = actions[a]
    r2, c2 = max(0,min(3,r+dr)), max(0,min(3,c+dc))
    s2 = 4*r2+c2
    reward = -1
    if s2==15: reward = 10
    return s2, reward

# Reward shaping potencial
phi = np.array([-(abs(r-3)+abs(c-3)) for r in range(4) for c in range(4)])

def shaping(s, s2):
    return gamma*phi[s2]-phi[s]

# Q-learning com reward shaping
alpha=0.1
eps=0.1
episodes=500

Q = np.zeros((n_states,n_actions))

for ep in range(episodes):
    s = np.random.randint(n_states)
    while s not in terminal:
        if np.random.rand()<eps:
            a = np.random.randint(n_actions)
        else:
            a = np.argmax(Q[s])
        s2, r = step(s,a)
        r_shaped = r + shaping(s,s2)
        Q[s,a] += alpha*(r_shaped + gamma*np.max(Q[s2]) - Q[s,a])
        s=s2

print("Política aprendida (melhor ação por estado):")
policy = np.array([np.argmax(Q[s]) for s in range(n_states)]).reshape(4,4)
print(policy)


Política aprendida (melhor ação por estado):
[[0 1 1 2]
 [1 1 1 2]
 [1 1 1 2]
 [1 1 1 0]]


A tabela acima mostra as melhores ações aprendidas em cada célula (0=up,1=right,2=down,3=left).

## Parte 2: Actor-Critic (A2C) em CartPole

In [2]:
pip install stable-baselines3 gymnasium[classic-control] -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import gymnasium as gym
from stable_baselines3 import A2C

# Criar ambiente CartPole
env = gym.make("CartPole-v1")

# Treinar A2C
model = A2C("MlpPolicy", env, verbose=0)
model.learn(total_timesteps=10000)

# Avaliar
obs, info = env.reset()
rewards = []
for _ in range(10):
    done=False
    total=0
    obs, info = env.reset()
    while not done:
        action, _ = model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        total += reward
        done = terminated or truncated
    rewards.append(total)

print("Recompensas médias em 10 episódios:", np.mean(rewards))


/home/ebezerra/anaconda3/envs/gcc1734/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Recompensas médias em 10 episódios: 203.6


O modelo A2C deve alcançar uma boa pontuação média após poucas iterações, demonstrando a efetividade dos métodos actor-critic.